In [1]:
import numpy as np
import math as m
np.set_printoptions(linewidth=np.inf)
from Matr import klocal,kglobal, orgElements,kglobal2f2,solveTruss,plotResults

In [8]:
#Project Inputs
areas = np.array([[12,12,12,12,12,12,12,12,12,12,12]])
coords = np.array([[0,0],[120,0],[240,0],[360,0],[480,0],[180,-800],[300,-800]])
con = np.array([[1,2],[2,3],[3,4],[4,5],[1,6],[2,6],[3,6],[3,7],[4,7],[5,7],[6,7]])
moduli = np.array([[29,29,29,29,29,29,29,29,29,29,29]])*1000
Bc = np.array([[1,1,0],[1,2,0],[5,1,0],[5,2,0]])
P = np.array([[2,2,-20],[3,2,-35],[4,2,-15]])
stcostLF = 2
stweight = 500/(12*1000)
n6 = 6
n7 = 7
jump1 = 0.25
jump2 = 0.5
n1 = 2
n2 = 3
n3 = 4
iterations = 500
y6base = 1
y7base = 2
nt = 3

def parseResults(table):
    size = np.shape(table)[0]
    result = np.zeros((3,1))
    baseVal = 100000000000
    for i in range(size):
        currentVal = table[i,0]
        if (currentVal < baseVal):
            baseVal = currentVal

    return baseVal
        

def ShellRun(Bc,P,coords, areas, moduli,con,n6,n7,y6base,y7base,iterations,steelcostLF,steelweight,jump1,jump2,n1,n2,n3,nt): 
    R_collection = np.zeros((iterations,3))
    for i in range(iterations):
        start6 = y6base
        start7 = y7base
        coordsM = Placeiter(coords,start6,start7)
        results = solveTruss(Bc,P,coordsM, areas, moduli, con)
        dispYall = results[7]
        lengths = results[8]
        stresses = results[9]
        dispY =  PullDisp(dispYall,n1,n2,n3,nt)

        memNum = np.shape(stresses)[0]

        num = 1
        for j in range(memNum):
            if (stresses[j] < 50):
                num = num + 1

        if (num > memNum):
            S = calcCost(dispY,lengths,steelcostLF,steelweight)
        else: 
            S = 1000000000001

        R_collection[i,0] = S
        R_collection[i,1] = start6
        R_collection[i,2] = start7
        y6base = y6base + jump1
        y7base = y7base + jump2

    return R_collection

def PullDisp(dispYall,n1,n2,n3,nt):
    cont = np.zeros((nt,1))
    cont[0,0] = dispYall[n1-1,0]
    cont[1,0] = dispYall[n2-1,0]
    cont[2,0] = dispYall[n3-1,0]
    return cont


def Placeiter(coords,y6,y7): 
    coords[n6-1,1] = y6
    coords[n7-1,1] = y7
    return coords

def calcCost(deflections,lengths,steelcostLF,steelweight):
    totalLengths = np.sum(lengths)
    totalWeight = steelweight * totalLengths
    C = steelcostLF * totalWeight
    sumDeflections = np.sum(deflections)
    S = np.absolute(sumDeflections) * C
    return S

table = ShellRun(Bc,P,coords, areas, moduli,con,n6,n7,y6base,y7base,iterations,stcostLF,stweight,jump1,jump2,n1,n2,n3,nt)
bestConfig = parseResults(table)

print(table)
print(bestConfig)



[[1.00000000e+12 1.00000000e+00 2.00000000e+00]
 [1.00000000e+12 1.25000000e+00 2.50000000e+00]
 [1.00000000e+12 1.50000000e+00 3.00000000e+00]
 ...
 [5.47733443e+01 1.25250000e+02 2.50500000e+02]
 [5.50976479e+01 1.25500000e+02 2.51000000e+02]
 [5.50976479e+01 1.25750000e+02 2.51500000e+02]]
53.171658014768276
